In [1]:
import json
import mysql.connector
from datetime import datetime
import time
from flask import Flask, jsonify

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="twitter"
)
cursor = mydb.cursor()

In [ ]:
#USER
user_id = set()
with open("corona-out-3", "r") as f:
    for line in f:
        try:            
            data = json.loads(line)
            user = data["user"]
            id = user['id']
            name = user['name']
            screen_name = user['screen_name']
            created_at = datetime.strptime(user['created_at'], '%a %b %d %H:%M:%S %z %Y')
            created_at = created_at.strftime('%Y-%m-%d %H:%M:%S')
            followers_count = user['followers_count']
            location = user['location']
            
            if id not in user_id:
                sql = "INSERT INTO users (id, name, screen_name, created_at, followers_count, location) VALUES (%s, %s, %s, %s, %s, %s)"
                val = (id, name, screen_name, created_at, followers_count, location)
                cursor.execute(sql, val)
                mydb.commit()
                user_id.add(id)
            
        except:
            continue

In [ ]:
#TWEET
tweet_id = set()
with open("corona-out-3", "r") as f:
    for line in f:
        try: 
            tweet = json.loads(line)
            if "retweeted_status" not in tweet:
                id = tweet['id']
                tweet_text = tweet['text']
                user_id = tweet['user']['id']
                quote_count = tweet['quote_count']
                reply_count = tweet['reply_count']
                retweet_count = tweet['retweet_count']
                favorite_count = tweet['favorite_count']
                created_at = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
                created_at = created_at.strftime('%Y-%m-%d %H:%M:%S')       
                language = tweet['lang']
            else:
                id = tweet['retweeted_status']['id']
                tweet_text = tweet['retweeted_status']['text']
                user_id = tweet['retweeted_status']['user']['id']
                quote_count = tweet['retweeted_status']['quote_count']
                reply_count = tweet['retweeted_status']['reply_count']
                retweet_count = tweet['retweeted_status']['retweet_count']
                favorite_count = tweet['retweeted_status']['favorite_count']
                created_at = datetime.strptime(tweet['retweeted_status']['created_at'], '%a %b %d %H:%M:%S %z %Y')
                created_at = created_at.strftime('%Y-%m-%d %H:%M:%S')       
                language = tweet['retweeted_status']['lang']
                
            if id not in tweet_id:
                sql = "INSERT INTO tweets (id, tweet_text, user_id, quote_count, reply_count, retweet_count, favorite_count, created_at, language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
                val = (id, tweet_text, user_id, quote_count, reply_count, retweet_count, favorite_count, created_at, language)
                cursor.execute(sql, val)
                mydb.commit()
                tweet_id.add(id)
                
        except:
            continue

In [ ]:
#RETWEET
with open("corona-out-3", "r") as f:
    for line in f:
        try:
            tweet = json.loads(line)
            if "retweeted_status" in tweet:
                retweet_id = tweet['id']
                user_id = tweet['user']['id']
                parent_tweet_id = tweet['retweeted_status']['id']
                parent_user_id = tweet['retweeted_status']['user']['id']
                created_at = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
                created_at = created_at.strftime('%Y-%m-%d %H:%M:%S')
                
                sql = "INSERT INTO retweets (retweet_id, user_id, parent_tweet_id, parent_user_id, created_at) VALUES (%s, %s, %s, %s, %s)"
                val = (retweet_id, user_id, parent_tweet_id, parent_user_id, created_at)
                cursor.execute(sql, val)
                mydb.commit()
        except:
            continue

In [ ]:
#Hashtags
with open("corona-out-3", "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            if "retweeted_status" in data:
                for i in data['retweeted_status']['extended_tweet']['entities']['hashtags']:
                    hashtag_text = i['text']
                    created_at = datetime.strptime(data['retweeted_status']['created_at'], '%a %b %d %H:%M:%S %z %Y')
                    created_at = created_at.strftime('%Y-%m-%d %H:%M:%S')
                    tweet_id = data['retweeted_status']['id']

                    sql = "INSERT INTO hashtags (hashtag_text, created_at, tweet_id) VALUES (%s, %s, %s)"
                    val = (hashtag_text, created_at, tweet_id)
                    cursor.execute(sql, val)
                    mydb.commit()
                    
        except:
            continue

In [ ]:
#Average Likes Per Tweet

app = Flask(__name__)

@app.route('/users/Popular-Hashtags/<string:name1>/<string:name2>', methods=['GET'])
def get_user(name1, name2):
    try:
        query = "SELECT hashtag_text, COUNT(*) as count FROM hashtags WHERE DATE(created_at) >= %s AND DATE(created_at) < %s GROUP BY hashtag_text ORDER BY count DESC LIMIT 10;"
        cursor.execute(query, (name1,name2))
        result = cursor.fetchall()

        # Create a list of dictionaries representing the data
        data = []
        for row in result:
            item = {
                'hashtag_text': row[0],
                'count': int(row[1]),
            }
            data.append(item)

        return jsonify(data)
    except Exception as e:
        # Handle exceptions
        return jsonify({'error': str(e)})

# Run the Flask application
if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2023 16:12:10] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2023 16:12:15] "GET /users/Popular-Hashtags/2020-04-01/2020-05-01 HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 16:12:24] "GET /users/Popular-Hashtags/2020-04-01/2020-05-01 HTTP/1.1" 200 -


In [ ]:
#Top 10 users
start_time = time.time()

query = "select * from users order by followers_count DESC LIMIT 10;"
cursor.execute(query)

for row in cursor.fetchall():
    print(row)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

In [ ]:
#Test
hashtags = {}
with open("corona-out-3", "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            if "retweeted_status" in data:
                for i in data['retweeted_status']['extended_tweet']['entities']['hashtags']:
                    if i['text'] not in hashtags:
                        hashtags[i['text']] = 1
                    else:
                        hashtags[i['text']] += 1

        except:
            continue

In [ ]:
len(hashtags)

In [ ]:
#Unique users
users = set()
total_tweets = 0
retweet_cnt = 0
with open("corona-out-3", "r") as f:
    for line in f:
        try:            
            data = json.loads(line)
            if "retweeted_status" in data:
                retweet_cnt += 1
            user_id = data["user"]["id"]
            users.add(user_id)
            total_tweets += 1
        except:
            continue

In [ ]:
print(len(users))

In [ ]:
total_tweets

In [ ]:
retweet_cnt

In [ ]:
#Original Tweets: 51K
#Unique Hashtags: 6.7K
#Unique Users: 81K
#Retweets: 61K